In [7]:
import json
datasets =None
with open ('datasets5.json', 'r') as f:
    datasets = json.load(f)
min_len = 10000
for dataset in datasets:
    node_token_list = dataset['node_token_list']
    if len(node_token_list) == 4:
        print(dataset['filename'])
print(min_len) 

60_data.txt
10000
